In [1]:
import os
import numpy as np
import pandas
import re
import urllib
from bs4 import BeautifulSoup as bs
from datetime import datetime as dt
import scipy.stats
from IPython.display import display, HTML

debug = 2
print('ok')

ok


In [2]:
os.chdir('/home/idies/workspace/Storage/raddick/raddick_folksonomy/data')
music_df = pandas.read_csv('cleaned_audio_data.csv', index_col='ID')
music_df = music_df.rename(columns={'Source': 'source', 'Sample File Name': 'filename'})

# Everything after the final period is the filename
if (debug >= 2):
    print('Parsing filename into stem and filetype...')
music_df['filetype'] = music_df['filename'].apply(lambda x: x.split('.')[-1])

# Delete stray .cue file
if (debug >= 2):
    print('Deleting stray .cue file...')
music_df = music_df.drop(music_df.index[music_df['filetype'] == 'cue'], axis=0)

# Get ready to score
music_df['score'] = -9999

# Everything before the final period is the stem
music_df['stem'] = music_df['filename'].apply(lambda x: x[0:len(x)-len(x.split('.')[-1])-1])

music_df['songonly'] = False

cd_like = re.compile('(?i)^[\s\-\._]*(br\-|dis|cd|sample|side|track|tit[le|el]|t[\d.])[\s\-\._]*')
onlynumbers = re.compile('^[0-9\.]+$')
intro_like = re.compile('(?i)^\d+[\s\.\-_\(\[]+\w*(intro|arriv|crowd|audience|stage|applause|tuning|announce|welcome|banter)')
unknown_like = re.compile('(?i)^\d*[\s\.\-_\(\[]*(unknown|artist|title)')
song_track_like = re.compile('(?i)^(d|\d)+[\s\.\-_\(\[\w]*(track)[\s\-\._]*\d+')
otherpatternfinder = re.compile('(?i)\d+\s*dr\d+')

music_df.loc[music_df['stem'].apply(lambda x: re.search(cd_like, x)).notnull(), 'songonly'] = True
music_df.loc[music_df['stem'].apply(lambda x: re.search(onlynumbers, x)).notnull(), 'songonly'] = True
music_df.loc[music_df['stem'].apply(lambda x: re.search(intro_like, x)).notnull(), 'songonly'] = True
music_df.loc[music_df['stem'].apply(lambda x: re.search(unknown_like, x)).notnull(), 'songonly'] = True
music_df.loc[music_df['stem'].apply(lambda x: re.search(song_track_like, x)).notnull(), 'songonly'] = True
music_df.loc[music_df['stem'].apply(lambda x: re.search(otherpatternfinder, x)).notnull(), 'songonly'] = True


music_df.loc[music_df['songonly'] , 'score'] = 0

print('music_df has {0:,.0f} total rows.'.format(len(music_df)))

print('{0:,.0f} rows set to score = 0.'.format(len(music_df[music_df['score'] == 0])))

music_df = music_df.drop('songonly', axis=1)

music_df_bk = music_df
print('ok')

#music_df['stem'][
#    (music_df['songonly']) 
#].sort_values()[125:]

Parsing filename into stem and filetype...
Deleting stray .cue file...
music_df has 2,972 total rows.
594 rows set to score = 0.
ok


In [3]:
music_df = music_df_bk

if (debug >= 2):
    print('Parsing stem into band and post-band...')
music_df.loc[music_df['stem'].apply(lambda x: re.search('\d', x)).notnull(), 'band_abbrev_autoparsed'] = music_df['stem'][music_df['stem'].apply(lambda x: re.search('\d', x)).notnull()].apply(lambda x: x[0:re.search('\d',x).start()])
music_df['postbandstuff'] = music_df['stem'][music_df['stem'].apply(lambda x: re.search('\d+', x)).notnull()].apply(lambda x: x[re.search('\d', x).start():len(x)])

# If no digits found, everything is part of the band_abbrev_autoparsed
if (debug >= 2):
    print('Fixing band values for stems with no digits...')

music_df['band_abbrev_autoparsed']  = music_df['band_abbrev_autoparsed'].fillna(music_df['stem'])
music_df['postbandstuff']  = music_df['postbandstuff'].fillna('')

# Fix the band U2
if (debug >= 2):
    print('Fixing band values for U2...')


music_df.loc[
    music_df['band_abbrev_autoparsed'].apply(lambda x: x.lower() == 'u')
    , 'band_abbrev_autoparsed'
] = music_df['stem'][
    music_df['band_abbrev_autoparsed'].apply(lambda x: x.lower() == 'u')
].apply(lambda x: x[0:re.search('2[\s_-]*', x).end()])

music_df.loc[
    music_df['band_abbrev_autoparsed'].apply(lambda x: re.search('u2', x.lower())).notnull()
    , 'postbandstuff'
] = music_df['stem'][
    music_df['band_abbrev_autoparsed'].apply(lambda x: re.search('u2', x.lower())).notnull()
].apply(lambda x: x[re.search('2[\s_-]*', x).end():len(x)])

         
music_df_bk = music_df

print('ok')

Parsing stem into band and post-band...
Fixing band values for stems with no digits...
Fixing band values for U2...
ok


In [4]:
music_df = music_df_bk
#                    ].dropna().drop_duplicates().sort_values()[0:30]#.values.tolist()
bandendspacers = re.compile('[\W_]+$')

music_df['hasbandendspacers'] = True
music_df.loc[music_df['band_abbrev_autoparsed'].apply(lambda x: re.search(bandendspacers, x)).isnull(), 'hasbandendspacers'] = False

music_df['band_abbrev'] = music_df['band_abbrev_autoparsed']

# Remove trailing spaces from band_abbrev_orig to create band_abbrev_parsed
music_df.loc[
    (music_df['hasbandendspacers']) 
    , 'band_abbrev'] = music_df['band_abbrev_autoparsed'][
    (music_df['hasbandendspacers']) 
].apply(lambda x: x[0:re.search(bandendspacers, x).start()])

# Fix Loreley
music_df.loc[music_df['band_abbrev'].apply(lambda x: 'loreley' in x.lower()), 'band_abbrev'
            ] = music_df['band_abbrev'][music_df['band_abbrev'
                                                ].apply(lambda x: 'loreley' in x.lower())
                                       ].apply(lambda x: x[0:len(x)-7])
music_df_bk = music_df
print('ok')

ok


In [5]:
music_df = music_df_bk
# NEED TO FIX THESE BAND VALUES
#music_df['band_abbrev'][~music_df.index.isin(fixbandlist)
#                              ].dropna().drop_duplicates().sort_values()[775:]
# VALUES FOUND THAT WAY ARE LISTED IN fixbandist
fixbandlist = []
fixbandlist += ['567194.html', '126863.html', '566104.html', '566727.html', '566252.html', '567047.html']
fixbandlist += ['566973.html', '566208.html', '565954.html', '565868.html', '566685.html', '565793.html']
fixbandlist += ['566439.html', '565913.html', '0140111caca68f331e53d9651a3eadf6b7b0f999']
fixbandlist += ['566783.html', '567544.html', '566278.html', '566234.html', '566526.html', '566814.html']
fixbandlist += ['566136.html', '566798.html', '567140.html', '565934.html', '566810.html', '566896.html']
fixbandlist += ['565650.html', '566756.html', '566114.html', '567536.html', '566329.html', '567879.html']
fixbandlist += ['567120.html', '567402.html', '565891.html', '567592.html', '566264.html', '566219.html']
fixbandlist += ['566734.html', '587888.html', '567758.html', '567795.html', '127033.html', '566511.html']
fixbandlist += ['565961.html', '566789.html', '565503.html', '566127.html', '566155.html', '565852.html']
fixbandlist += ['127034.html', '566231.html', '566157.html', '566144.html', '566657.html', '567318.html']
fixbandlist += ['565923.html', '566150.html', '566066.html', '565901.html']
fixbandlist += ['567771.html', 'c2b7c10aa42aa6f55c8537ece72ed5726ae6b1b3', '566903.html']
fixbandlist += ['567234.html', '566050.html', '567202.html', '567760.html', '587769.html', '567336.html']
# Write the rows that need to be fixed into a CSV, fix them in XLSX document
#music_df[['band_abbrev']][music_df.index.isin(fixbandlist)].sort_values(by='band_abbrev').to_csv('bandstofix.csv')

# Load the uploaded Excel document with the fixes
fixed_bands_df = pandas.read_excel('bandstofix.xlsx', index_col='ID')

# Perform band fixes
music_df = music_df.join(fixed_bands_df, how='left')
music_df.loc[music_df['band_abbrev_fixed'].notnull(), 'band_abbrev'] = music_df['band_abbrev_fixed']
music_df = music_df.drop('band_abbrev_fixed', axis=1)

# Some more bands to fix
music_df.loc['567466.html', 'band_abbrev'] = 'sales'
music_df.loc['565676.html', 'band_abbrev'] = 'Sweet Liberties'
music_df.loc['567731.html', 'band_abbrev'] = 'The Brew'
music_df.loc['565549.html', 'band_abbrev'] = 'BC'
music_df.loc['567361.html', 'band_abbrev'] = 'Coral Creek'


# Find blank values of band_abbrev that can be found from stem values and fixed
#music_df['stem'][
#    (~music_df['songonly']) &
#    (music_df['band_abbrev'] == '')
#].sort_values()[775:800]
addbandlist = []
addbandlist += ['566895.html', '566989.html', '567507.html', '567718.html', '566324.html', '566687.html']
addbandlist += ['567464.html', '567692.html', '565610.html', '566238.html', '565714.html', '566937.html']
addbandlist += ['566938.html', '567634.html', '566723.html', '03bcddfb8e4c6e4df25f285517205919e41bca35']
addbandlist += ['127059.html', '566930.html', '566932.html', '566655.html', '565529.html', '567192.html']
addbandlist += ['567503.html', '565750.html', '566261.html', '567810.html', '565970.html', '567294.html']
addbandlist += ['566280.html', '566679.html', 'f704e684bd9817702d272c0d06cd118d695b0517', '565958.html']
addbandlist += ['567845.html', '566294.html', '566673.html', '566674.html', '566805.html', '566804.html']
addbandlist += ['566854.html', '566726.html', '566748.html', '566793.html', '566873.html', '566485.html']
addbandlist += ['565929.html', '567431.html', '565966.html', '566629.html']
addbandlist += ['567201.html', '567025.html', '567022.html', '567150.html', '567088.html']
addbandlist += ['567285.html', '565948.html', '566371.html', '566119.html', '567146.html', '566724.html']
addbandlist += ['566879.html', '567566.html', '567368.html', '567271.html']
addbandlist += ['126776.html', '127051.html', '567639.html', '567323.html', '126935.html', '127156.html']
addbandlist += ['566933.html', '567830.html', '565959.html', '566017.html', '567417.html']
addbandlist += ['567653.html', '566851.html', '567233.html', '567496.html', '566488.html']
addbandlist += ['566106.html', 'f984d4ab17b0a85de1bfa9ffab80efd7625b8122', '567338.html']
addbandlist += ['567717.html', '567578.html', '566890.html', '566546.html', '565723.html']
addbandlist += ['3c39e32bf0918ec3190ecda08442eba0d8b2faa8', '567064.html', '566799.html', '567275.html']
addbandlist += ['567442.html', '587360.html', '565748.html', '587570.html', '587699.html']
addbandlist += ['587637.html', '566375.html', '587700.html', '587634.html', '587693.html']
addbandlist += ['587632.html', '126991.html', '126990.html', '567166.html', '127071.html']
addbandlist += ['567240.html', '567775.html', '567053.html', '565602.html']
#music_df['stem'][music_df.index.isin(addbandlist)].sort_values().to_csv('bandstoadd.csv')

# Load the uploaded Excel document with the fixes
added_bands_df = pandas.read_excel('bandstoadd.xlsx', index_col='ID')

# Add new bands
music_df = music_df.join(added_bands_df, how='left')
music_df.loc[music_df['band_abbrev_new'].notnull(), 'band_abbrev'] = music_df['band_abbrev_new']
music_df = music_df.drop('band_abbrev_new', axis=1)

# Lower and strip band named parsed
music_df['band_abbrev'] = music_df['band_abbrev'].apply(lambda x: x.lower().strip())

music_df_bk = music_df
print('ok')

ok


In [6]:
# GET LIST OF OFFICIAL ABBREVIATIONS

print('Retrieving from etree')
pageurl = 'http://wiki.etree.org/index.php?page=BandAbbreviations'
page = urllib.request.urlopen(pageurl).read()
soup = bs(page, "html.parser")
tables = soup.find_all('table')

#for thistable in tables:
#    print(thistable)
#    print('---------')

bandtable = tables[1]
bands = bandtable.find_all('tr')
allbands = []

print('Thinking...')
for thisband in bands[1:]:
    theirdata = thisband.find_all('td')
    oneband = {}
    oneband['name'] = theirdata[0].text
    oneband['abbrev'] = theirdata[1].text
    try:
        oneband['otherabbrev'] = theirdata[2].text
    except:
        pass
    try:
        oneband['notes'] = theirdata[3].text
    except:
        pass
    allbands.append(oneband)
abbrev_df = pandas.DataFrame.from_records(allbands)
abbrev_df.index.name = 'rownumber'

abbrev_df = abbrev_df.drop('notes', axis=1)
column_order = ['name', 'abbrev', 'otherabbrev']
abbrev_df = abbrev_df[column_order]

# Trim one space from beginning and end of band abbrev and name
abbrev_df['abbrev'] = abbrev_df['abbrev'].apply(lambda x: str(x).strip())
abbrev_df['name'] = abbrev_df['name'].apply(lambda x: str(x).strip())

# Trim away brackets from beginning and end of band name (for some bands)
abbrev_df['name'] = abbrev_df['name'] = np.where((abbrev_df['name'].apply(lambda x: (str(x)[0])) == '['), 
                             abbrev_df['name'].apply(lambda x: str(x)[1:]), 
                             abbrev_df['name']) 
abbrev_df['name'] = abbrev_df['name'] = np.where((abbrev_df['name'].apply(lambda x: (str(x)[len(str(x))-1])) == ']'),
                              abbrev_df['name'].apply(lambda x: str(x)[:-1]), 
                              abbrev_df['name'])
# dumb special cases
abbrev_df.loc[abbrev_df['abbrev'] == '(see FB below)', 'abbrev'] = ''
abbrev_df.loc[abbrev_df['abbrev'] == 'Keller Williams w- String Cheese Incident', 'abbrev'] = 'kwi'


# lower and strip, fillna w/blanks for otherabbrev
abbrev_df['abbrev'] = abbrev_df['abbrev'].apply(lambda x: str(x).lower().strip())
abbrev_df['otherabbrev'] = abbrev_df['otherabbrev'].fillna('')
abbrev_df['otherabbrev'] = abbrev_df['otherabbrev'].apply(lambda x: str(x).lower().strip())

abbrev_df_bk = abbrev_df
print('ok')

Retrieving from etree
Thinking...
ok


In [7]:
# ROTATE ABBREVIATIONS INTO GUIDE FILE
guide_df = pandas.DataFrame(data=abbrev_df['abbrev'][abbrev_df['abbrev'] != ''], columns=['abbrev'])

guide_df = guide_df.merge(abbrev_df, how='left', on='abbrev')
guide_df = guide_df[['abbrev', 'name']]
guide_df = guide_df.set_index('abbrev')
guide_df['canonical'] = True

variant_abbrev_list = []
for thisrow in abbrev_df['otherabbrev'][abbrev_df['otherabbrev'] != ''].drop_duplicates().tolist():
    for thisabbrev in thisrow.split(','):
        thisabbrev = thisabbrev.strip()
        thisband = abbrev_df['name'][abbrev_df['otherabbrev'].apply(lambda x: thisabbrev in x)].values[0]
        if thisabbrev not in guide_df.index.values:
            guide_df.loc[thisabbrev] = [thisband, False]

guide_df = guide_df.sort_values('canonical').sort_index()

# change index value of kdub
indexaslist = guide_df.index.tolist()
kdubvalue = indexaslist.index('kdub (for k-double-u)')
indexaslist[kdubvalue] = 'kdub'
guide_df.index = indexaslist

# Clean up some weird-ass values
guide_df.loc['beanland', 'name'] = 'Beanland'
guide_df.loc['buho', 'canonical'] = True
guide_df.loc['logic', 'canonical'] = True
guide_df.loc['gtb', 'canonical'] = True
guide_df.loc['ig', 'canonical'] = True
guide_df.loc['jgb', 'name'] = 'Jerry Garcia Band'
guide_df.loc['jgb', 'canonical'] = True
guide_df.loc['jt', 'canonical'] = True
guide_df.loc['ls', 'canonical'] = True
guide_df.loc['los', 'canonical'] = True
guide_df.loc['marlow', 'canonical'] = True
guide_df.loc['moe', 'name'] = 'Moe'
guide_df.loc['pmb', 'canonical'] = True
guide_df.loc['amendola', 'canonical'] = True
guide_df.loc['hip', 'canonical'] = True
guide_df.loc['wilco', 'canonical'] = True
guide_df.loc['spod', 'name'] = 'Serial Pod'

guide_df.index.name = 'abbrev'

guide_df_bk = guide_df
guide_df.sample(3)

,name,canonical
abbrev,,
bog,A Band of Gypsys,True
gt,Ghost Trane,True
beanland,Beanland,True


In [8]:
music_df = music_df_bk

music_df['abbrev_type'] = 'notfound'
music_df.loc[music_df['band_abbrev'] == '', 'abbrev_type'] = 'blank'

music_df = music_df.reset_index().merge(guide_df.reset_index(), how='left', left_on='band_abbrev', right_on='abbrev').set_index('ID')
music_df = music_df.rename(columns={'name': 'band'})
music_df = music_df.drop('stem_old', axis=1)
music_df.loc[(music_df['canonical'].notnull()) & (music_df['canonical'] == True), 'abbrev_type'] = 'canonical'
music_df.loc[(music_df['canonical'].notnull()) & (music_df['canonical'] == False), 'abbrev_type'] = 'variant'

# Band names used incorrect abbreviations
incorrect = ['phish', 'widespreadpanic']
incorrect += ['gov`t mule', 'pl&amp;f', 'deadfeat']
music_df.loc[music_df['band_abbrev'].isin(incorrect), 'abbrev_type'] = 'incorrect'

# Filter out some non-bands by labeling them as "notband"
music_df.loc[music_df['band_abbrev'].apply(lambda x: 'dis' in x.lower()), 'abbrev_type'] = 'notband'
music_df.loc[music_df['band_abbrev'].apply(lambda x: 'track' in x.lower()), 'abbrev_type'] = 'notband'
music_df.loc[music_df['band_abbrev'].apply(lambda x: 'track' in x.lower()), 'abbrev_type'] = 'notband'
music_df.loc[music_df['band_abbrev'].apply(lambda x: 'artist' in x.lower()), 'abbrev_type'] = 'notband'
music_df.loc[music_df['band_abbrev'].apply(lambda x: 'cd ' in x.lower()), 'abbrev_type'] = 'notband'
music_df.loc[music_df['band_abbrev'] == 'cd', 'abbrev_type'] = 'notband'
music_df.loc[music_df['band_abbrev'].apply(lambda x: 'tit' in x.lower()), 'abbrev_type'] = 'notband'
music_df.loc[music_df['band_abbrev'].apply(lambda x: 'side ' in x.lower()), 'abbrev_type'] = 'notband'
music_df.loc[music_df['band_abbrev'].apply(lambda x: 'sample' in x.lower()), 'abbrev_type'] = 'notband'

music_df_bk = music_df

music_df.groupby('abbrev_type').size()
#music_df.head(1)

abbrev_type
blank        1019
canonical     350
incorrect      48
notband       225
notfound     1309
variant        21
dtype: int64

In [9]:
music_df = music_df_bk
bands_df = pandas.read_excel('bands_identified.xlsx')
#bands_df.tail(10)
music_df = music_df.reset_index().merge(bands_df, how='left', on='band_abbrev', suffixes=['', '_identified_name']).set_index('ID')
music_df.loc[music_df['band_identified_name'].notnull(), 'band'] = music_df['band_identified_name']
music_df.loc[music_df['band_identified_name'].notnull(), 'abbrev_type'] = 'assigned'
music_df = music_df.drop('band_identified_name', axis=1)
#music_df[music_df['abbrev_type'] == 'assigned']
music_df.loc[(music_df['abbrev_type'] == 'assigned') & (music_df['band'] == 'notband'), 'abbrev_type'] = 'notband'
music_df.loc[(music_df['abbrev_type'] == 'assigned') & (music_df['band'] == 'unknown'), 'abbrev_type'] = 'unknown'

#print('ok')
music_df_bk = music_df
print(music_df.groupby('abbrev_type').size())
#print(music_df.groupby('band').size().sort_values(ascending=False))

abbrev_type
assigned     1149
blank        1019
canonical     349
incorrect      48
notband       315
unknown        71
variant        21
dtype: int64


In [10]:
# FIND CASES WHERE WE PARSED A MONTH NAME IN WITH A BAND NAME
#music_df['band_abbrev_autoparsed'][
#    music_df['band_abbrev_autoparsed'].apply(
#        lambda x: 'dec' in x.lower())].apply(lambda x: '|'+x+'|').sort_values()

music_df['month_in_band_abbrev'] = ''

music_df.loc[music_df['band_abbrev_autoparsed'] == 'NickCave_Metro_Feb', ['band_abbrev', 'month_in_band_abbrev']] = ['nickcave', 'feb']
music_df.loc[music_df['band_abbrev_autoparsed'] == 'PDP_LoungeAx_Feb', ['band_abbrev', 'month_in_band_abbrev']] = ['pdp', 'feb']
music_df.loc[music_df['band_abbrev_autoparsed'] == 'The Jam Bolingbrook Mar ', ['band_abbrev', 'month_in_band_abbrev']] = ['pdp', 'mar']
music_df.loc[music_df['band_abbrev_autoparsed'] == 'EHG - April ', ['band_abbrev', 'month_in_band_abbrev']] = ['ehg', 'apr']
music_df.loc[music_df['band_abbrev_autoparsed'] == 'Bob Dylan - June ', ['band_abbrev', 'month_in_band_abbrev']] = ['bob dylan', 'jun']
music_df.loc[music_df['band_abbrev_autoparsed'] == 'Mavis Staples - June ', ['band_abbrev', 'month_in_band_abbrev']] = ['mavis staples', 'jun']
music_df.loc[music_df['band_abbrev_autoparsed'] == 'Casting Crowns Royals Stadium Kansas City Missouri July ', ['band_abbrev', 'month_in_band_abbrev']] = ['casting crowns', 'jul']
music_df.loc[music_df['band_abbrev_autoparsed'] == 'Demiricous - July ', ['band_abbrev', 'month_in_band_abbrev']] = ['demiricous', 'jul']
music_df.loc[music_df['band_abbrev_autoparsed'] == 'KISS Independence Mo Events Center July ', ['band_abbrev', 'month_in_band_abbrev']] = ['kiss', 'jul']
music_df.loc[music_df['band_abbrev_autoparsed'] == 'Paul McCartney - July ', ['band_abbrev', 'month_in_band_abbrev']] = ['paul mccartney', 'jul']
music_df.loc[music_df['band_abbrev_autoparsed'] == 'VAN MORRISON - Montreux July ', ['band_abbrev', 'month_in_band_abbrev']] = ['van morrison', 'jul']
music_df.loc[music_df['band_abbrev_autoparsed'] == 'VM - Hamburg July ', ['band_abbrev', 'month_in_band_abbrev']] = ['vm', 'jul']
music_df.loc[music_df['band_abbrev_autoparsed'] == 'Weird Al Yankovic - July ', ['band_abbrev', 'month_in_band_abbrev']] = ['weird al yankovic', 'jul']
music_df.loc[music_df['band_abbrev_autoparsed'] == 'VM - Birmingham September ', ['band_abbrev', 'month_in_band_abbrev']] = ['vm', 'sep']

print('{0:.0f} months parsed out of band abbrevs'.format(len(music_df[['filename', 'band_abbrev_autoparsed', 'band_abbrev', 'band', 'month_in_band_abbrev']][music_df['month_in_band_abbrev'] != ''])))
#music_df['band_abbrev_autoparsed'].loc['566252.html']

14 months parsed out of band abbrevs


In [11]:
music_df = music_df_bk
datefinder = re.compile('[\d]+[\s\-\._]+[\d]+[\s\-\._]+[\d]+')
yyyyfinder = re.compile('[\d]{4}-[\d]{2}-[\d]{2}')
yyfinder = re.compile('[\d]{2}-[\d]{2}-[\d]{2}')
date1 = re.compile('[\d]{2,4}[\s\.\-_]*[\d]{1,2}[\s\.\-_]*[\d]{1,2}')

music_df['thedate'] = pandas.to_datetime(music_df['postbandstuff'][music_df['postbandstuff'].apply(lambda x: re.search(datefinder, x)).notnull()].apply(lambda x: x[re.search(datefinder, x).start():re.search(datefinder, x).end()]), errors='coerce')

music_df['date_format'] = 'otros'

music_df.loc[(music_df['postbandstuff'].apply(lambda x: re.search(yyyyfinder, x)).notnull()), 'date_format'] = 'yyyy-mm-dd'
music_df.loc[(music_df['date_format'] == 'otros') & (music_df['postbandstuff'].apply(lambda x: re.search(yyfinder, x)).notnull()), 'date_format'] = 'yy-mm-dd'

music_df.loc[
    (music_df['thedate'].isnull()) & 
    (music_df['postbandstuff'].apply(lambda x: re.search(date1, x)).notnull())
    , 'thedate'] = pandas.to_datetime(music_df['postbandstuff'][(music_df['thedate'].isnull()) & (music_df['postbandstuff'].apply(lambda x: re.search(date1, x)).notnull())
                                                               ].apply(lambda x: x[re.search(date1, x).start():re.search(date1, x).end()]), errors='coerce')
music_df.loc[(music_df['date_format'] == 'otros') & (music_df['thedate'].notnull()), 'date_format'] = '2 or 4 digit year, separator, 1 or 2 digit month, separator, 1 or 2 digit day'

# Set up year and month for filling in by hand
music_df['theyear'] = np.nan
music_df['themonth'] = np.nan
music_df['theday'] = np.nan

# Fix some dates by hand
music_df.loc[['567662.html', '567649.html', '567750.html'], ['thedate', 'date_format']
            ] = [pandas.to_datetime('2016-07-23'), 'byhand']
music_df.loc[['565762.html', '565781.html'], ['thedate', 'date_format']
            ] = [pandas.to_datetime('2016-07-03'), 'byhand']
music_df.loc[['565838.html', '126834.html'], ['thedate', 'date_format']
            ] = [pandas.to_datetime('2016-07-02'), 'byhand']
music_df.loc[['565838.html', '126834.html'], ['thedate', 'date_format']
            ] = [pandas.to_datetime('2016-07-02'), 'byhand']
music_df.loc['566023.html', ['thedate', 'date_format']
            ] = [pandas.to_datetime('2016-07-08'), 'byhand']
music_df.loc['566144.html', ['thedate', 'date_format']
            ] = [pandas.to_datetime('1982-11-05'), 'byhand']
music_df.loc['566144.html', ['thedate', 'date_format']
            ] = [pandas.to_datetime('1982-11-05'), 'byhand']
music_df.loc['566231.html', ['thedate', 'date_format']
            ] = [pandas.to_datetime('1988-02-29'), 'byhand']
music_df.loc['566526.html', ['thedate', 'date_format']
            ] = [pandas.to_datetime('1978-11-13'), 'byhand']
music_df.loc['567038.html', ['thedate', 'date_format']
            ] = [pandas.to_datetime('2003-09-23'), 'byhand']
music_df.loc['567500.html', ['thedate', 'date_format']
            ] = [pandas.to_datetime('2016-07-26'), 'byhand']
music_df.loc[['567549.html', '127174.html'], ['thedate', 'date_format']
            ] = [pandas.to_datetime('2016-07-27'), 'byhand']
music_df.loc['566758.html', ['theyear', 'themonth', 'date_format']
            ] = [1991, 7, 'byhand']

# Discovered through regex search for [\d]{6}
music_df.loc['566210.html', ['thedate', 'date_format']] = [pandas.to_datetime('2016-01-08'), 'byhand']
music_df.loc['566223.html', ['thedate', 'date_format']] = [pandas.to_datetime('1988-07-09'), 'byhand']
music_df.loc['566252.html', ['thedate', 'date_format']] = [pandas.to_datetime('2016-07-09'), 'byhand']
music_df.loc['567003.html', ['thedate', 'date_format']] = [pandas.to_datetime('2016-07-17'), 'byhand']
music_df.loc['567140.html', ['thedate', 'date_format']] = [pandas.to_datetime('2016-07-20'), 'byhand']
music_df.loc['567671.html', ['thedate', 'date_format']] = [pandas.to_datetime('1989-11-19'), 'byhand']
music_df.loc['567872.html', ['thedate', 'date_format']] = [pandas.to_datetime('2016-07-30'), 'byhand']

# Month wrongly parsed as part of band name
music_df.loc[music_df['filename'] == 'NickCave_Metro_Feb18-1989_D6_909_T01.flac', ['thedate', 'date_format']] = [pandas.to_datetime('1989-02-18'), 'byhand']
music_df.loc[music_df['filename'] == 'VM - Hamburg July 5th 1997_Disc 02 Track 02.flac', ['thedate', 'date_format']] = [pandas.to_datetime('1997-07-05'), 'byhand']
music_df.loc[music_df['filename'] == 'EHG - April 29, 2016 - t01.flac', ['thedate', 'date_format']] = [pandas.to_datetime('2016-04-29'), 'byhand']
music_df.loc[music_df['filename'] == 'Weird Al Yankovic - July 7, 2016 - t01.flac', ['thedate', 'date_format']] = [pandas.to_datetime('2016-07-07'), 'byhand']
music_df.loc[music_df['filename'] == 'Bob Dylan - June 25, 2016 - t01.flac', ['thedate', 'date_format']] = [pandas.to_datetime('2016-06-25'), 'byhand']
music_df.loc[music_df['filename'] == 'Mavis Staples - June 25, 2016 - t01.flac', ['thedate', 'date_format']] = [pandas.to_datetime('2016-06-25'), 'byhand']
music_df.loc[music_df['filename'] == 'The Jam Bolingbrook Mar 7 1980 16.flac', ['thedate', 'date_format']] = [pandas.to_datetime('1980-03-07'), 'byhand']
music_df.loc[music_df['filename'] == 'PDP_LoungeAx_Feb17-1990_D6_909_T01.flac', ['thedate', 'date_format']] = [pandas.to_datetime('1990-02-17'), 'byhand']
music_df.loc[music_df['filename'] == 'Casting Crowns Royals Stadium Kansas City Missouri July 9th 201601.flac', ['thedate', 'date_format']] = [pandas.to_datetime('2016-07-09'), 'byhand']
music_df.loc[music_df['filename'] == 'Paul McCartney - July 10, 2016 - t01.flac', ['thedate', 'date_format']] = [pandas.to_datetime('2016-07-10'), 'byhand']
music_df.loc[music_df['filename'] == 'VAN MORRISON - Montreux July 7th 2016 (biddie)_Track 17.flac', ['thedate', 'date_format']] = [pandas.to_datetime('2016-07-07'), 'byhand']
music_df.loc[music_df['filename'] == 'Demiricous - July 15, 2016 - t01.flac', ['thedate', 'date_format']] = [pandas.to_datetime('2016-07-15'), 'byhand']

music_df.loc[music_df['filename'] == 'VM - Birmingham September 2nd 2011_Disc 01 Track 04.flac', ['thedate', 'date_format']] = [pandas.to_datetime('2011-09-02'), 'byhand']

# Discovered by looking through each value
music_df.loc['567291.html', ['thedate', 'date_format']] = [pandas.to_datetime('2000-02-25'), 'byhand']
music_df.loc['567291.html', ['themonth', 'theday', 'date_format']] = [7, 4, 'byhand']
music_df.loc['565561.html', ['thedate', 'date_format']] = [pandas.to_datetime('2011-09-06'), 'byhand']
music_df.loc['567156.html', ['thedate', 'date_format']] = [pandas.to_datetime('2014-04-26'), 'byhand']
music_df.loc['566417.html', ['thedate', 'date_format']] = [pandas.to_datetime('2013-03-08'), 'byhand']
music_df.loc['565929.html', ['thedate', 'date_format']] = [pandas.to_datetime('2006-03-01'), 'byhand']
music_df.loc['567639.html', ['thedate', 'date_format']] = [pandas.to_datetime('1975-01-03'), 'byhand']
music_df.loc['587454.html', ['thedate', 'date_format']] = [pandas.to_datetime('2016-01-30'), 'byhand']
music_df.loc['566234.html', ['thedate', 'date_format']] = [pandas.to_datetime('1979-11-18'), 'byhand']
music_df.loc['567376.html', ['thedate', 'date_format']] = [pandas.to_datetime('1977-08-01'), 'byhand']
music_df.loc['587339.html', ['thedate', 'date_format']] = [pandas.to_datetime('2016-02-19'), 'byhand']
music_df.loc['567790.html', ['thedate', 'date_format']] = [pandas.to_datetime('2017-07-26'), 'byhand']
music_df.loc['567790.html', ['thedate', 'date_format']] = [pandas.to_datetime('2016-07-22'), 'byhand']
music_df.loc['566157.html', ['thedate', 'date_format']] = [pandas.to_datetime('1979-11-24'), 'byhand']
music_df.loc['567572.html', ['thedate', 'date_format']] = [pandas.to_datetime('2016-07-22'), 'byhand']
music_df.loc['565793.html', ['thedate', 'date_format']] = [pandas.to_datetime('1981-06-27'), 'byhand']
music_df.loc['587337.html', ['thedate', 'date_format']] = [pandas.to_datetime('2016-06-03'), 'byhand']
music_df.loc['587588.html', ['thedate', 'date_format']] = [pandas.to_datetime('2016-07-10'), 'byhand']
music_df.loc['587640.html', ['thedate', 'date_format']] = [pandas.to_datetime('2016-07-08'), 'byhand']
music_df.loc['587572.html', ['thedate', 'date_format']] = [pandas.to_datetime('2016-07-09'), 'byhand']
music_df.loc['126880.html', ['thedate', 'date_format']] = [pandas.to_datetime('1988-10-01'), 'byhand']
music_df.loc['567487.html', ['themonth', 'theday', 'date_format']] = [8, 14, 'byhand']
music_df.loc['566266.html', ['themonth', 'theday', 'date_format']] = [11, 24, 'byhand']
music_df.loc['565981.html', ['themonth', 'theday', 'date_format']] = [8, 29, 'byhand']
music_df.loc['567478.html', ['theyear', 'themonth', 'date_format']] = [2000, 6, 'byhand']
music_df.loc['567323.html', ['theyear', 'date_format']] = [1983, 'byhand']
music_df.loc['567526.html', ['themonth', 'theday', 'date_format']] = [10, 13, 'byhand']
music_df.loc['566219.html', ['theyear', 'date_format']] = [1990, 'byhand']
music_df.loc['565891.html', ['theyear', 'date_format']] = [1989, 'byhand']
music_df.loc['566208.html', ['theyear', 'themonth', 'date_format']] = [1972, 1, 'byhand']
music_df.loc['e059e8d39da4b75145b1e434b0569df9c8f59367', ['theyear', 'date_format']] = [1995, 'byhand']
music_df.loc['567222.html', ['themonth', 'theday', 'date_format']] = [2, 11, 'byhand']
music_df.loc['566756.html', ['theyear', 'themonth', 'date_format']] = [2011, 1, 'byhand']
music_df.loc['566756.html', ['theyear', 'themonth', 'date_format']] = [2016, 1, 'byhand']
music_df.loc['566756.html', ['theyear', 'themonth', 'date_format']] = [1980, 7, 'byhand']


music_df.loc[(music_df['theyear'].isnull()) & (music_df['thedate'].notnull())
    , 'theyear'] = music_df['thedate'][(music_df['theyear'].isnull()) & (music_df['thedate'].notnull())
                                      ].apply(lambda x: x.year)
music_df.loc[(music_df['themonth'].isnull()) & (music_df['thedate'].notnull())
    , 'themonth'] = music_df['thedate'][(music_df['themonth'].isnull()) & (music_df['thedate'].notnull())
                                      ].apply(lambda x: x.month)
music_df.loc[(music_df['theday'].isnull()) & (music_df['thedate'].notnull())
    , 'theday'] = music_df['thedate'][(music_df['theday'].isnull()) & (music_df['thedate'].notnull())
                                      ].apply(lambda x: x.day)

music_df.loc[music_df['date_format'] == 'otros', 'date_format'] = 'nodate'

music_df_bk = music_df
print(music_df.groupby('date_format').size())


date_format
2 or 4 digit year, separator, 1 or 2 digit month, separator, 1 or 2 digit day     190
byhand                                                                             64
nodate                                                                           1408
yy-mm-dd                                                                          131
yyyy-mm-dd                                                                       1179
dtype: int64


In [12]:
seps = re.compile('[\s\-\.\/\\_]')

music_df = music_df_bk

music_df['date_right_after_band'] = False
music_df.loc[
    (music_df['band_abbrev_autoparsed'] != '') &
    ((music_df['band_abbrev_autoparsed'].apply(lambda x: re.search(seps, x[-1:])).isnull())) &
    (music_df['postbandstuff'] != '') &
    (music_df['postbandstuff'].apply(lambda x: re.search('\d', x[0:])).notnull()) &
    (music_df['thedate'].notnull() | music_df['theyear'].notnull() | music_df['themonth'].notnull() | music_df['theday'].notnull())
    ,'date_right_after_band'] = True

music_df_bk = music_df
music_df.groupby('date_right_after_band').size()

date_right_after_band
False    1966
True     1006
dtype: int64

In [13]:
#music_df.loc[(music_df[''])]
music_df = music_df_bk

# Full 4 points for: 
#    * band abbrev is canonical or assigned (correct but not in etree's list)
#    * date directly follows band with no space between
#    * date is correctly formatted as yyyy-mm-dd

music_df.loc[
    (music_df['abbrev_type'].apply(lambda x: x in ['canonical', 'assigned'])) & 
    (music_df['date_format'] == 'yyyy-mm-dd') & 
    (music_df['date_right_after_band']), 'score'] = 4

# Score goes down to 3 if...
#    date is formatted as yy-mm-dd, OR
music_df.loc[
    (music_df['abbrev_type'].apply(lambda x: x in ['canonical', 'assigned'])) & 
    (music_df['date_format'] == 'yy-mm-dd') & 
    (music_df['date_right_after_band']), 'score'] = 3

#    date does not directly follow band, OR
music_df.loc[
    (music_df['abbrev_type'].apply(lambda x: x in ['canonical', 'assigned'])) & 
    (music_df['date_format'] == 'yyyy-mm-dd') & 
    (~music_df['date_right_after_band']), 'score'] = 3

#    they use a variant abbreviation
music_df.loc[
    (music_df['abbrev_type'].apply(lambda x: x in ['variant'])) & 
    (music_df['date_format'] == 'yyyy-mm-dd') & 
    (music_df['date_right_after_band']), 'score'] = 4

# Score goes down to 2 if...
#    date is present and readable but formatted strangely, OR
music_df.loc[
    (music_df['abbrev_type'].apply(lambda x: x in ['canonical', 'assigned', 'variant'])) & 
    (music_df['date_format'].apply(lambda x: 
                                   x in ['2 or 4 digit year, separator, 1 or 2 digit month, separator, 1 or 2 digit day', 'byhand']))\
    , 'score'] = 2

#    if the band has a correct abbreviation but they used an incorrect one
music_df.loc[
    (music_df['abbrev_type'].apply(lambda x: x in ['incorrect'])) & 
    (music_df['date_format'].apply(lambda x: 
                                   x in ['yyyy-mm-dd', 'yy-mm-dd']))
    , 'score'] = 2

# Score = 0 if there is no band AND no date
music_df.loc[
    (music_df['abbrev_type'].apply(lambda x: x in ['blank', 'notband'])) & 
    (music_df['date_format'] == 'nodate'), 'score'] = 0

# OTHERWISE, score = 1
music_df.loc[music_df['score'] == -9999, 'score'] = 1

music_df_bk = music_df
#music_df.groupby('score').size()
#music_df[['filename', 'source', 'filetype', 'band_abbrev', 'abbrev_type', 'band', 'thedate', 'date_format', 'date_right_after_band', 'score']][
music_df['filename'][music_df['score'] ==  0].sample(1)

ID
565729.html    01 Hells Bells.flac
Name: filename, dtype: object

In [14]:
music_df[['filename', 'source', 'filetype', 'band_abbrev', 'abbrev_type', 'band', 'thedate', 'date_format', 'date_right_after_band', 'score']
        ].sort_values(by='score', ascending=False).to_csv('outmusic.csv')
print('ok')

ok


In [15]:
small_sample_cutoff_value = 4

music_df['band'] = music_df['band'].fillna('blank')

stats_allbands_df = music_df.groupby('band')['score'].describe()

stats_df = stats_allbands_df[stats_allbands_df['count'] >= small_sample_cutoff_value]

miscbandlist = stats_allbands_df[~stats_allbands_df.index.isin(stats_df.index)].index.values.tolist()

miscband_df = pandas.DataFrame(data=['All other bands'], columns=['band'])
for thiscol in stats_df.columns:
    miscband_df[thiscol] = np.nan

miscband_df['count'] = music_df[music_df['band'].isin(miscbandlist)]['score'].count()
miscband_df['mean'] = music_df[music_df['band'].isin(miscbandlist)]['score'].mean()
miscband_df['std'] = music_df[music_df['band'].isin(miscbandlist)]['score'].std()
miscband_df['min'] = music_df[music_df['band'].isin(miscbandlist)]['score'].min()
miscband_df['25%']= music_df[music_df['band'].isin(miscbandlist)]['score'].quantile(0.25)
miscband_df['50%']= music_df[music_df['band'].isin(miscbandlist)]['score'].quantile(0.5)
miscband_df['75%']= music_df[music_df['band'].isin(miscbandlist)]['score'].quantile(0.75)
miscband_df['max'] = music_df[music_df['band'].isin(miscbandlist)]['score'].max()
miscband_df = miscband_df.set_index('band')

stats_df = stats_df.append(miscband_df)

print('ok')

ok


In [16]:
stats_df = stats_df.sort_values(by='mean', ascending=False)

newindex = stats_df[~stats_df.index.isin(['All other bands', 'unknown', 'notband', 'blank'])].index
othertypeslist = ['All other bands', 'unknown', 'notband', 'blank']
othertypesindex = pandas.Index(othertypeslist)
newindex = newindex.append(othertypesindex)
stats_df = stats_df.reindex(newindex)

htmlstring = '<table>'
#htmlstring += '<tr><th>Band</th><th>Count (n)</th><th>Mean (M)</th><th>Standard deviation (S)</th><th>Minimum</th><th>25th percentile</th><th>Median</th><th>75th percentile</th><th>Maximum</th></tr>'
htmlstring += '<tr><th>Band</th><th>Count (n)</th><th>Mean (M)</th><th>Standard deviation (S)</th>'
for thisband, thisrow in stats_df[['count', 'mean', 'std']].iterrows():
    htmlstring += '<tr>'
    htmlstring += '<td>{:}</td>'.format(thisband)
    htmlstring += '<td>{:,.0f}</td>'.format(thisrow['count'])
    htmlstring += '<td>{:.1f}</td>'.format(thisrow['mean'])
    htmlstring += '<td>{:.2f}</td>'.format(thisrow['std'])
    htmlstring += '</tr>'
htmlstring += '</table>'        
display(HTML(htmlstring))

Band,Count (n),Mean (M),Standard deviation (S)
Big Smith,5,4.0,0.00
Dean Monkey & The Dropouts,15,4.0,0.00
Head for the Hills,4,4.0,0.00
Raccoon,4,4.0,0.00
String Cheese Incident,4,4.0,0.00
Gov't Mule,4,4.0,0.00
The Pixies,4,4.0,0.00
TBC,4,4.0,0.00
Dave Matthews Band,9,4.0,0.00
Chris Robinson Brotherhood,11,4.0,0.00


In [17]:
music_df = music_df_bk

jambands_df = pandas.read_csv('jambands/jambands.csv')

music_df = music_df.reset_index().merge(jambands_df, on='band', how='left').set_index('ID')
music_df.loc[(music_df['band'].apply(lambda x: x not in ['blank', 'notband', 'unknown'])) & (music_df['jamband'].isnull()), 'jamband'] = 'no'
music_df['jamband'] = music_df['jamband'].fillna('n/a')

jamband_stats_df = music_df.groupby('jamband')['score'].describe()

newindex = pandas.Index(['yes', 'maybe', 'no', 'n/a'])
jamband_stats_df = jamband_stats_df.reindex(newindex)

htmlstring = '<table>'
#htmlstring += '<tr><th>Band</th><th>Count (n)</th><th>Mean (M)</th><th>Standard deviation (S)</th><th>Minimum</th><th>25th percentile</th><th>Median</th><th>75th percentile</th><th>Maximum</th></tr>'
htmlstring += '<tr><th>Jam Band?</th><th>Filenames (n)</th><th>Mean (M)</th><th>Standard deviation (S)</th>'
for thisband, thisrow in jamband_stats_df[['count', 'mean', 'std']].iterrows():
    htmlstring += '<tr>'
    htmlstring += '<td>{:}</td>'.format(thisband)
    htmlstring += '<td>{:,.0f}</td>'.format(thisrow['count'])
    htmlstring += '<td>{:.1f}</td>'.format(thisrow['mean'])
    htmlstring += '<td>{:.2f}</td>'.format(thisrow['std'])
    htmlstring += '</tr>'
htmlstring += '</table>'        
display(HTML(htmlstring))

Jam Band?,Filenames (n),Mean (M),Standard deviation (S)
yes,405,3.5,0.65
maybe,19,3.6,0.77
no,"1,108",3.0,1.03
n/a,"1,453",0.2,0.43


In [18]:
htmlstring = '<h2>t-tests!</h2>'


htmlstring += '<h3>yes vs. no</h3>'

htmlstring += '<table>'
htmlstring += '<tr><th>Jam band?</th><th>Mean (M)</th><th>St Dev (S)</th><th>Count (n)</th></tr>'

for g in ['yes', 'no']:
    htmlstring += '<td>{0:}</td><td>{1:.1f}</td><td>{2:.2f}</td><td>{3:,.0f}</td></tr>'.format(
         g,
         jamband_stats_df['mean'].loc[g], 
         jamband_stats_df['std'].loc[g], 
         jamband_stats_df['count'].loc[g])
#y
htmlstring += '</table>'
display(HTML(htmlstring))

group1 = music_df['score'][music_df['jamband'] == 'yes']
group2 = music_df['score'][music_df['jamband'] == 'no']

t, p = scipy.stats.ttest_ind(group1, group2)
print('t = {0:.3f}, p = {1:.3f}'.format(t, p))


htmlstring = '<h3>yes vs. maybe</h3>'

htmlstring += '<table>'
htmlstring += '<tr><th>Jam band?</th><th>Mean (M)</th><th>St Dev (S)</th><th>Count (n)</th></tr>'

for g in ['yes', 'maybe']:
    htmlstring += '<td>{0:}</td><td>{1:.1f}</td><td>{2:.2f}</td><td>{3:,.0f}</td></tr>'.format(
         g,
         jamband_stats_df['mean'].loc[g], 
         jamband_stats_df['std'].loc[g], 
         jamband_stats_df['count'].loc[g])
htmlstring += '</table>'
display(HTML(htmlstring))

group1 = music_df['score'][music_df['jamband'] == 'yes']
group2 = music_df['score'][music_df['jamband'] == 'maybe']

t, p = scipy.stats.ttest_ind(group1, group2)
print('t = {0:.3f}, p = {1:.3f}'.format(t, p))



Jam band?,Mean (M),St Dev (S),Count (n),yes,3.5,0.65,405,no,3.0,1.03,"1,108"


t = 10.038, p = 0.000


Jam band?,Mean (M),St Dev (S),Count (n),yes,3.5,0.65,405,maybe,3.6,0.77,19


t = -0.247, p = 0.805


## Conclusion: We can consider yes and maybe together.

In [19]:
music_df = music_df_bk
music_df['band'] = music_df['band'].fillna('blank')

jambands_df = pandas.read_csv('jambands/jambands.csv')
jambands_df.loc[jambands_df['jamband'] == 'maybe', 'jamband'] = 'yes'

music_df = music_df.reset_index().merge(jambands_df, on='band', how='left').set_index('ID')
music_df.loc[(music_df['band'].apply(lambda x: x not in ['blank', 'notband', 'unknown'])) & (music_df['jamband'].isnull()), 'jamband'] = 'no'
music_df['jamband'] = music_df['jamband'].fillna('n/a')
jamband_stats_df = music_df.groupby('jamband')['score'].describe()

newindex = pandas.Index(['yes', 'no', 'n/a'])
jamband_stats_df = jamband_stats_df.reindex(newindex)
htmlstring = '<table>'
#htmlstring += '<tr><th>Band</th><th>Count (n)</th><th>Mean (M)</th><th>Standard deviation (S)</th><th>Minimum</th><th>25th percentile</th><th>Median</th><th>75th percentile</th><th>Maximum</th></tr>'
htmlstring += '<tr><th>Jam Band?</th><th>Filenames (n)</th><th>Mean (M)</th><th>Standard deviation (S)</th>'
for thisband, thisrow in jamband_stats_df[['count', 'mean', 'std']].iterrows():
    htmlstring += '<tr>'
    htmlstring += '<td>{:}</td>'.format(thisband)
    htmlstring += '<td>{:,.0f}</td>'.format(thisrow['count'])
    htmlstring += '<td>{:.1f}</td>'.format(thisrow['mean'])
    htmlstring += '<td>{:.2f}</td>'.format(thisrow['std'])
    htmlstring += '</tr>'
htmlstring += '</table>'        
display(HTML(htmlstring))

Jam Band?,Filenames (n),Mean (M),Standard deviation (S)
yes,424,3.5,0.66
no,"1,108",3.0,1.03
n/a,"1,453",0.2,0.43


In [20]:
htmlstring = '<h2>t-tests!</h2>'


htmlstring += '<h3>yes vs. no</h3>'

htmlstring += '<table>'
htmlstring += '<tr><th>Jam band?</th><th>Mean (M)</th><th>St Dev (S)</th><th>Count (n)</th></tr>'

for g in ['yes', 'no']:
    htmlstring += '<td>{0:}</td><td>{1:.1f}</td><td>{2:.2f}</td><td>{3:,.0f}</td></tr>'.format(
         g,
         jamband_stats_df['mean'].loc[g], 
         jamband_stats_df['std'].loc[g], 
         jamband_stats_df['count'].loc[g])
#y
htmlstring += '</table>'
display(HTML(htmlstring))

group1 = music_df['score'][music_df['jamband'] == 'yes']
group2 = music_df['score'][music_df['jamband'] == 'no']

t, p = scipy.stats.ttest_ind(group1, group2)
print('t = {0:.3f}, p = {1:.3f}'.format(t, p))

Jam band?,Mean (M),St Dev (S),Count (n),yes,3.5,0.66,424,no,3.0,1.03,"1,108"


t = 10.263, p = 0.000


In [21]:
music_df = music_df_bk
music_df['band'] = music_df['band'].fillna('blank')

jambands_df = pandas.read_csv('jambands/jambands.csv')
jambands_df.loc[jambands_df['jamband'] == 'maybe', 'jamband'] = 'yes'
music_df = music_df.reset_index().merge(jambands_df, on='band', how='left').set_index('ID')
music_df.loc[(music_df['band'].apply(lambda x: x not in ['blank', 'notband', 'unknown'])) & (music_df['jamband'].isnull()), 'jamband'] = 'no'
music_df['jamband'] = music_df['jamband'].fillna('n/a')

small_sample_cutoff_value = 4

stats_allbands_df = music_df.groupby('band')['score'].describe()

stats_df = stats_allbands_df[stats_allbands_df['count'] >= small_sample_cutoff_value]

miscbandlist = stats_allbands_df[~stats_allbands_df.index.isin(stats_df.index)].index.values.tolist()

#print('ok')
music_df.loc[(music_df['jamband'] == 'no') & (~music_df['band'].isin(miscbandlist)), 'jamband'] = 'popular no'
music_df.loc[(music_df['jamband'] == 'no') & (music_df['band'].isin(miscbandlist)), 'jamband'] = 'rare no'

music_df['jamband'] = music_df['jamband'].fillna('n/a')
jamband_stats_df = music_df.groupby('jamband')['score'].describe()

newindex = pandas.Index(['yes', 'popular no', 'rare no', 'n/a'])
jamband_stats_df = jamband_stats_df.reindex(newindex)
htmlstring = '<table>'
#htmlstring += '<tr><th>Band</th><th>Count (n)</th><th>Mean (M)</th><th>Standard deviation (S)</th><th>Minimum</th><th>25th percentile</th><th>Median</th><th>75th percentile</th><th>Maximum</th></tr>'
htmlstring += '<tr><th>Jam Band?</th><th>Filenames (n)</th><th>Mean (M)</th><th>Standard deviation (S)</th>'
for thisband, thisrow in jamband_stats_df[['count', 'mean', 'std']].iterrows():
    htmlstring += '<tr>'
    htmlstring += '<td>{:}</td>'.format(thisband)
    htmlstring += '<td>{:,.0f}</td>'.format(thisrow['count'])
    htmlstring += '<td>{:.1f}</td>'.format(thisrow['mean'])
    htmlstring += '<td>{:.2f}</td>'.format(thisrow['std'])
    htmlstring += '</tr>'
htmlstring += '</table>'        
display(HTML(htmlstring))

Jam Band?,Filenames (n),Mean (M),Standard deviation (S)
yes,424,3.5,0.66
popular no,368,3.1,1.02
rare no,740,2.9,1.04
n/a,"1,453",0.2,0.43


In [22]:
htmlstring = '<h2>t-tests!</h2>'


htmlstring += '<h3>popular no vs. rare no</h3>'

htmlstring += '<table>'
htmlstring += '<tr><th>Jam band?</th><th>Mean (M)</th><th>St Dev (S)</th><th>Count (n)</th></tr>'

for g in ['popular no', 'rare no']:
    htmlstring += '<td>{0:}</td><td>{1:.1f}</td><td>{2:.2f}</td><td>{3:,.0f}</td></tr>'.format(
         g,
         jamband_stats_df['mean'].loc[g], 
         jamband_stats_df['std'].loc[g], 
         jamband_stats_df['count'].loc[g])
#y
htmlstring += '</table>'

display(HTML(htmlstring))

group1 = music_df['score'][music_df['jamband'] == 'popular no']
group2 = music_df['score'][music_df['jamband'] == 'rare no']

t, p = scipy.stats.ttest_ind(group1, group2)
print('t = {0:.3f}, p = {1:.3f}'.format(t, p))


htmlstring = '<h3>yes vs. popular no</h3>'

htmlstring += '<table>'
htmlstring += '<tr><th>Jam band?</th><th>Mean (M)</th><th>St Dev (S)</th><th>Count (n)</th></tr>'

for g in ['yes', 'popular no']:
    htmlstring += '<td>{0:}</td><td>{1:.1f}</td><td>{2:.2f}</td><td>{3:,.0f}</td></tr>'.format(
         g,
         jamband_stats_df['mean'].loc[g], 
         jamband_stats_df['std'].loc[g], 
         jamband_stats_df['count'].loc[g])
#y
htmlstring += '</table>'
display(HTML(htmlstring))


group1 = music_df['score'][music_df['jamband'] == 'yes']
group2 = music_df['score'][music_df['jamband'] == 'popular no']

t, p = scipy.stats.ttest_ind(group1, group2)
print('t = {0:.3f}, p = {1:.3f}'.format(t, p))


Jam band?,Mean (M),St Dev (S),Count (n),popular no,3.1,1.02,368,rare no,2.9,1.04,740


t = 2.181, p = 0.029


Jam band?,Mean (M),St Dev (S),Count (n),yes,3.5,0.66,424,popular no,3.1,1.02,368


t = 7.601, p = 0.000
